#TEST

In [1]:
%load_ext autoreload
%autoreload 1
%aimport WKP 
#import nbimporter 
import numpy as np
import pandas as pd
import os

from ipywidgets import widgets
#debug view
debug_view = widgets.Output(layout={'border': '1px solid black'})

WKP.verbose=True
lokaties=WKP.load_lokatie()    
oordeel=WKP.load_oordeel()
WKP.check_lokaties(oordeel,lokaties)
orig_oordeelset=WKP.merge_data(oordeel,lokaties)
#doelen=WKP.load_doelen()
#WKP.check_lokaties(doelen,lokaties)
#doelenset=WKP.merge_data(doelen,lokaties)
print('ready loading data')


LOKATIES: datarecords: 4032 without duplicates: 925 (waterbodies: 925)


,waardebepalingsmethode_code,waardebepalingsmethode_omschrijving,rapportagejaar
5388,AQUOKIT,Met Aquo-kit berekend oordeel of toetsresultaat,2009
0,AQUOKIT,Met Aquo-kit berekend oordeel of toetsresultaat,2014
0,AQUOKIT,Met Aquo-kit berekend oordeel of toetsresultaat,2015
0,AQUOKIT,Met Aquo-kit berekend oordeel of toetsresultaat,2016
0,AQUOKIT,Met Aquo-kit berekend oordeel of toetsresultaat,2017
0,AQUOKIT,Met Aquo-kit berekend oordeel of toetsresultaat,2018
0,DESK,Bepaling klasse door deskundige,2009
20,DESK,Bepaling klasse door deskundige,2014
39886,DESK,Bepaling klasse door deskundige,2015
5902,DESK,Bepaling klasse door deskundige,2016


renaming overige waterflora -> overige waterflora-kwaliteit
renaming overige relevante verontreinigende stoffen -> Specifieke verontreinigende stoffen
OORDEEL: datarecords: 618465 without duplicates: 614520 (waterbodies: 965)
  CHECK:waterbodies missing in lokaties, present in dataset
  MERGE: datarecords: 611167 without duplicates: 611167 (waterbodies: 925)
['2009' '2014' '2015' '2016' '2017' '2018']
ready loading data


<IPython.core.display.Javascript object>

In [50]:
#preselections on dataset
#create an ordered list, be aware that NaNs could be present ->dropna
stoffen=orig_oordeelset['typering_omschrijving'].dropna().unique()
stoffen=sorted(stoffen)
oordeelset=orig_oordeelset

out = widgets.Output(layout={'border': '1px solid blue'})

stofselectie = widgets.Dropdown(
    options=stoffen,
    value=stoffen[0],
    description='Selectie:',
)

groupselector=widgets.RadioButtons(
    options=['algemeen', 'chemie'],
    value='algemeen',
    description='Selectie :',
    disabled=False
)

methodselector=widgets.RadioButtons(
    options=['TT_OM', 'TT','OM'],
    value='TT_OM',
    description='Methode :',
    disabled=False
)

SGselector=widgets.RadioButtons(
    options=['all', 'NLEM','NLMS','NLRN','NLSC'],
    value='all',
    description='Stroomgebied:',
    disabled=False
)
bepalingselector=widgets.RadioButtons(
    options=['all','AQUOKIT', 'DESK'],
    value='all',
    description='Bepaling:',
    disabled=False
)
gebiedselectie = widgets.Dropdown(
    options=['stroomgebied','waterschap'],
    value='stroomgebied',
    description='Gebiedsnivo:',
)
wtypes=sorted(oordeelset['waterlichaam_type'].dropna().unique())


typeselectie = widgets.Dropdown(
    options=['all']+wtypes,
    value='all',
    description='Waterlichaamtype:',
)
#checkboxes 
checkwaterbeheer=widgets.Checkbox(
    value=False,
    description='Restrict to waterbeheerder',
    disabled=False
)

showtable=widgets.Checkbox(
    value=False,
    description='Show table with graphs',
    disabled=False
)


#main routine
@debug_view.capture(clear_output=True)
def changeview():
    subset=oordeelset[(oordeelset['oordeelsoort_code']==methodselector.value)]
        
    if bepalingselector.value!='all':
           subset=subset[(subset['waardebepalingsmethode_code']==bepalingselector.value)]
    if typeselectie.value!='all':
           subset=subset[(subset['waterlichaam_type']==typeselectie.value)]
    if SGselector.value!='all':
        subset=subset[(subset['stroomgebieddistrict_code']==SGselector.value)]
    
    if stofselectie.value!='all':
        if groupselector.value!='chemie':
          subset=subset[(subset.typering_omschrijving==stofselectie.value)]
        else:
          subset=subset[(subset.chemischestof_omschrijving==stofselectie.value)]  
           
    with out:
        if subset.shape[0]==0:
           print('No data available')
           return
        
        if gebiedselectie.value=='stroomgebied':
           WKP.display_stroomgebied_oordeel(subset, showtable.value, stofselectie.value)
        if gebiedselectie.value=='waterschap':
           WKP.display_waterbeheerder_oordeel(subset,  showtable.value,  stofselectie.value)
        

def selector_change(change):
    if groupselector.value!='chemie':
        stoffen=oordeelset['typering_omschrijving'].dropna().unique()
    else:
        stoffen=oordeelset['chemischestof_omschrijving'].dropna().unique()
    
    stoffen=sorted(stoffen)
    stofselectie.options=stoffen

def response(change):
       out.clear_output(wait=True)
       changeview()
    
#SELECTOR RESPONSE      
stofselectie.observe(response, names="value")
typeselectie.observe(response, names="value")
gebiedselectie.observe(response, names="value")
groupselector.observe(selector_change, names="value" )
methodselector.observe(response, names="value" )
SGselector.observe(response, names="value" )
bepalingselector.observe(response, names="value" )
showtable.observe(response, names="value")
checkwaterbeheer.observe(response, names="value" )

#VISUAL
containerleft = widgets.VBox([groupselector, stofselectie, gebiedselectie, typeselectie, showtable])
containerright = widgets.VBox([methodselector, bepalingselector])
containermid = widgets.VBox([SGselector])
widgetcontainer = widgets.HBox([containerleft,containermid, containerright])
fullcontainer = widgets.VBox([widgetcontainer,debug_view,out])
display(fullcontainer)
changeview()
